In [1]:
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
msles_df1 = pd.read_csv("store_nbr_family_best_msles2.csv")
msles_df2 = pd.read_csv("top_down_store_nbr_family_best_msles2.csv")
msles_df3 = pd.read_csv("top_down_family_then_store_nbr_best_msles2.csv")

In [3]:
cutoffs = msles_df1.columns[2:]

In [4]:
msles_df = msles_df1.merge(msles_df2, on=["store_nbr", "family"])
for cutoff in cutoffs:
    msles_df[cutoff] = (msles_df[[cutoff + "_x", cutoff + "_y"]]).min(axis=1)
    msles_df = msles_df.drop([cutoff + "_x", cutoff + "_y"], axis=1)
msles_df.head()

,store_nbr,family,cluster,2017-05-08,2017-06-05,2017-07-03,2017-07-31
0,1,AUTOMOTIVE,13,0.135487,0.155650,0.319663,0.249930
1,1,BABY CARE,13,0.000000,0.000000,0.000000,0.000000
2,1,BEAUTY,13,0.133388,0.172049,0.193954,0.175956
3,1,BEVERAGES,13,0.036529,0.018366,0.016777,0.019927
4,1,BOOKS,13,0.129721,0.231963,0.134613,0.029989


In [5]:
msles_df = msles_df.merge(msles_df3, on=["store_nbr", "family"])
for cutoff in cutoffs:
    msles_df[cutoff] = (msles_df[[cutoff + "_x", cutoff + "_y"]]).min(axis=1)
    msles_df = msles_df.drop([cutoff + "_x", cutoff + "_y"], axis=1)
msles_df.head()

,store_nbr,family,cluster,2017-05-08,2017-06-05,2017-07-03,2017-07-31
0,1,AUTOMOTIVE,13,0.135487,0.155650,0.319663,0.249930
1,1,BABY CARE,13,0.000000,0.000000,0.000000,0.000000
2,1,BEAUTY,13,0.133388,0.172049,0.193954,0.175956
3,1,BEVERAGES,13,0.036529,0.018366,0.016777,0.019927
4,1,BOOKS,13,0.129721,0.231963,0.134613,0.027932


In [6]:
msles_df.iloc[:, 3:].mean()**0.5

2017-05-08    0.405977
2017-06-05    0.390097
2017-07-03    0.382396
2017-07-31    0.383456
dtype: float64